# Lyric Analyzer

## 1. Prepare dataset
### 1. Open dataset
This dataset was downloaded from
https://www.kaggle.com/artimous/every-song-you-have-heard-almost

In [1]:
import pandas as pd
import numpy as np

df1 = pd.read_csv('../dataset/Lyrics1.csv')
df2 = pd.read_csv('../dataset/Lyrics2.csv')
df = df1.append(df2)
#release memory
del df1, df2

df

,Band,Lyrics,Song
0,Elijah Blake,"No, no\r\nI ain't ever trapped out the bando\r...",Everyday
1,Elijah Blake,"The drinks go down and smoke goes up, I feel m...",Live Till We Die
2,Elijah Blake,She don't live on planet Earth no more\r\nShe ...,The Otherside
3,Elijah Blake,"Trippin' off that Grigio, mobbin', lights low\...",Pinot
4,Elijah Blake,"I see a midnight panther, so gallant and so br...",Shadows & Diamonds
5,Elijah Blake,I just want to ready your mind\r\n'Cause I'll ...,Uno
6,Elijah Harris,To believe\r\nOr not to believe\r\nThat is the...,Girlfriend (Main)
7,Elijah Levi,"No one here can love or understand me\r\nOh, w...",Bye Bye Blackbird
8,Elijah Levi,"Lullaby of Birdland, that's what I \r\nAlways ...",Lullaby of Birdland
9,Elijah Levi,I hate to see that evening sun go down\r\nI ha...,St. Louis Blues


In [2]:
artists = df.Band.unique()
print("Unique artists: ", artists.size)

#http://cmdlinetips.com/2018/01/how-to-add-a-new-column-to-using-a-dictionary-in-pandas-data-frame/

Unique artists:  42627


### 2. Tag artists with genres
To get the genres of every artist use Spotify api at https://api.spotify.com/v1/.

Save a csv file with two columns:
* Band: the name of the band
* Genres: an array with the artist's genres

In [4]:
%%script skip
import requests
token = 'BQBWR7j8rLSM3lTkFafP3DDO0VjzsLWVtQg2rL5LdQm-5jCZu30_XKOCrS5YmdiXUQ3f73P24Y26uYqQAPxloEnG2I1XkMIATMRKqGk2NkD-p7Cb8W4VyY8pToJGzA8lgVOpxtDgi6fYUJtLgSGfG4eFFQ'
url = 'https://api.spotify.com/v1/search'
headers = {'Accept': 'application/json', 'Content-Type':'application/json', 'Authorization':'Bearer '+token}

artist_genres = []
artist_no_genres = set([])

for i in range(32676, artists.size):#artists.size
    if(i%10 == 0):
        print("\r{0}".format(i), end="\r")
    artist=artists[i]
    params = {'q': artist, 'type': 'artist', 'limit':'1'}   
    response = requests.get(url, params=params, headers=headers)
    if(response.ok):
        jData= response.json()
        if(jData['artists']['items'] and jData['artists']['items'][0]['genres']):
            a = {}
            a['name'] = artist
            a['genres'] = jData['artists']['items'][0]['genres']
            artist_genres.append(a)
            pd.DataFrame([a]).to_csv('../dataset/artists.csv', index=False, mode='a', header=False)
        else:
            artist_no_genres.add(artist)
    else:
        print("Crashed at index ", i)
        response.raise_for_status()
        
print("Artists found: ", len(artist_genres))
print("Artists not found: ", len(artist_no_genres))

Couldn't find program: 'skip'


### 3. Delete non english lyrics


In [7]:
from langdetect import detect
from langdetect import DetectorFactory 
DetectorFactory.seed = 0

lyrics_dropped = 0
for row in df.itertuples(index=True, name='Pandas'):
    lyric = getattr(row, "Lyrics")
    index = getattr(row, "Index")
    try:
        if detect(lyric) != 'en':
            df.drop(index, inplace=True)
            lyrics_dropped += 1
    except:
        pass

print("Lyrics dropped ", lyrics_dropped)
df.to_csv('../dataset/Lyrics_en.csv', index=False)
df

Lyrics dropped  54261


,Band,Lyrics,Song
0,Elijah Blake,"No, no\r\nI ain't ever trapped out the bando\r...",Everyday
1,Elijah Blake,"The drinks go down and smoke goes up, I feel m...",Live Till We Die
2,Elijah Blake,She don't live on planet Earth no more\r\nShe ...,The Otherside
3,Elijah Blake,"Trippin' off that Grigio, mobbin', lights low\...",Pinot
4,Elijah Blake,"I see a midnight panther, so gallant and so br...",Shadows & Diamonds
5,Elijah Blake,I just want to ready your mind\r\n'Cause I'll ...,Uno
6,Elijah Harris,To believe\r\nOr not to believe\r\nThat is the...,Girlfriend (Main)
7,Elijah Levi,"No one here can love or understand me\r\nOh, w...",Bye Bye Blackbird
8,Elijah Levi,"Lullaby of Birdland, that's what I \r\nAlways ...",Lullaby of Birdland
9,Elijah Levi,I hate to see that evening sun go down\r\nI ha...,St. Louis Blues
